# 📦 CELLULE 1 : Imports et Configuration

In [ ]:
"""
╔══════════════════════════════════════════════════════════════════════════╗
║                   MNIST CNN - Classification de Chiffres                 ║
║                                                                          ║
║  Objectif : Entraîner un CNN performant pour classifier les chiffres     ║
║             manuscrits avec optimisation bayésienne des hyperparamètres  ║
╚══════════════════════════════════════════════════════════════════════════╝
"""

# ============================================================================
# IMPORTS
# ============================================================================

# Librairies de base
import numpy as np
import pandas as pd
import json
import time
from pathlib import Path

# PyTorch - Deep Learning
import torch
import torch.nn as nn
import torch.optim as optim

# Visualisation
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches

# Métriques et évaluation
from sklearn.metrics import (
    roc_auc_score, 
    confusion_matrix,
    classification_report, 
    silhouette_score,
    davies_bouldin_score
)

# Réduction de dimensionnalité
from sklearn.manifold import TSNE

# Optimisation bayésienne
import optuna
from optuna.visualization import plot_param_importances, plot_optimization_history

# ============================================================================
# CONFIGURATION
# ============================================================================

# Détection automatique du device (GPU si disponible, sinon CPU)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"🖥️  Device configuré : {device}")
if torch.cuda.is_available():
    print(f"   GPU détecté : {torch.cuda.get_device_name(0)}")
    print(f"   Mémoire disponible : {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")

# Seed pour reproductibilité
RANDOM_SEED = 42
torch.manual_seed(RANDOM_SEED)
np.random.seed(RANDOM_SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed(RANDOM_SEED)

# ============================================================================
# CHEMINS DES FICHIERS
# ============================================================================

# Adapter selon ton environnement (Google Colab vs local)
DRIVE_PATH = Path('/content/drive/MyDrive/La Plateforme/Handwritten Digits')
DATA_TRAIN_PATH = DRIVE_PATH / 'mnist_train.csv'
DATA_TEST_PATH = DRIVE_PATH / 'mnist_test.csv'
MODEL_PATH = DRIVE_PATH / 'model_final_CNN.pth'
HYPERPARAMS_PATH = DRIVE_PATH / 'best_hyperparams_CNN.json'
GRAPH_DIR = DRIVE_PATH / 'graphs'

# Créer le dossier graphs s'il n'existe pas
GRAPH_DIR.mkdir(parents=True, exist_ok=True)

print("✅ Configuration chargée avec succès")
print(f"📁 Répertoire de travail : {DRIVE_PATH}")


# 📊 CELLULE 2 : Chargement des Données

In [ ]:
"""
╔══════════════════════════════════════════════════════════════════════════╗
║                         CHARGEMENT DES DONNÉES                           ║
╚══════════════════════════════════════════════════════════════════════════╝

Dataset MNIST : 60,000 images d'entraînement + 10,000 images de test
Format : Images 28×28 pixels en niveaux de gris (0-255)
Classes : 10 chiffres (0 à 9)
"""

def load_mnist_data_cnn(train_path, test_path, train_split=0.8, shuffle=True):
    """
    Charge et prépare les données MNIST pour un CNN.
    
    Le CNN nécessite des données au format : (batch, channels, height, width)
    Pour MNIST : (N, 1, 28, 28) où N = nombre d'exemples
    
    Args:
        train_path (str|Path): Chemin vers mnist_train.csv
        test_path (str|Path): Chemin vers mnist_test.csv
        train_split (float): Proportion pour le training set (le reste = dev set)
        shuffle (bool): Mélanger les données avant le split
    
    Returns:
        tuple: (X_train, Y_train, X_dev, Y_dev, X_test, Y_test)
               - X : Tensors de shape (N, 1, 28, 28), normalisés [0, 1]
               - Y : Tensors de labels (N,) de type long
    
    Example:
        >>> X_train, Y_train, X_dev, Y_dev, X_test, Y_test = load_mnist_data_cnn(
        ...     'mnist_train.csv', 'mnist_test.csv'
        ... )
        >>> print(X_train.shape)  # torch.Size([48000, 1, 28, 28])
    """
    
    print("\n" + "="*70)
    print("📊 CHARGEMENT DES DONNÉES MNIST POUR CNN")
    print("="*70)
    
    # ========================================================================
    # ÉTAPE 1 : Charger les fichiers CSV
    # ========================================================================
    
    print("\n🔄 Chargement des fichiers CSV...")
    data_train_full = pd.read_csv(train_path).values
    data_test_full = pd.read_csv(test_path).values
    
    print(f"   ✓ Train brut : {data_train_full.shape[0]:,} exemples")
    print(f"   ✓ Test : {data_test_full.shape[0]:,} exemples")
    
    # ========================================================================
    # ÉTAPE 2 : Mélanger les données (optionnel)
    # ========================================================================
    
    if shuffle:
        print("\n🔀 Mélange des données d'entraînement...")
        np.random.shuffle(data_train_full)
    
    # ========================================================================
    # ÉTAPE 3 : Split Train/Dev
    # ========================================================================
    
    split_idx = int(train_split * len(data_train_full))
    data_train = data_train_full[:split_idx]
    data_dev = data_train_full[split_idx:]
    
    print(f"\n📂 Split {int(train_split*100)}/{int((1-train_split)*100)} :")
    print(f"   ✓ Train : {len(data_train):,} exemples")
    print(f"   ✓ Dev : {len(data_dev):,} exemples")
    
    # ========================================================================
    # ÉTAPE 4 : Préparer les tenseurs pour CNN
    # ========================================================================
    
    def prepare_cnn_data(data):
        """
        Convertit données brutes en tensors PyTorch format CNN.
        
        Transformations :
        1. Extraire labels (dernière colonne)
        2. Extraire pixels (toutes les colonnes sauf dernière)
        3. Normaliser pixels : [0, 255] → [0, 1]
        4. Reshape : (N, 784) → (N, 1, 28, 28)
        5. Envoyer sur GPU/CPU
        """
        # Labels
        labels = torch.tensor(data[:, -1], dtype=torch.long)
        
        # Images : reshape (N, 784) → (N, 1, 28, 28)
        images = torch.tensor(data[:, :-1], dtype=torch.float32)
        images = images.reshape(-1, 1, 28, 28)
        
        # Normalisation [0, 255] → [0, 1]
        images = images / 255.0
        
        return images.to(device), labels.to(device)
    
    print("\n⚙️  Conversion en tensors PyTorch...")
    X_train, Y_train = prepare_cnn_data(data_train)
    X_dev, Y_dev = prepare_cnn_data(data_dev)
    X_test, Y_test = prepare_cnn_data(data_test_full)
    
    # ========================================================================
    # ÉTAPE 5 : Résumé final
    # ========================================================================
    
    print("\n" + "="*70)
    print("✅ DONNÉES CHARGÉES ET PRÊTES")
    print("="*70)
    print(f"{'Dataset':<10} {'Shape':<25} {'Device':<10} {'Dtype'}")
    print("-" * 70)
    print(f"{'X_train':<10} {str(tuple(X_train.shape)):<25} {str(X_train.device):<10} {X_train.dtype}")
    print(f"{'Y_train':<10} {str(tuple(Y_train.shape)):<25} {str(Y_train.device):<10} {Y_train.dtype}")
    print(f"{'X_dev':<10} {str(tuple(X_dev.shape)):<25} {str(X_dev.device):<10} {X_dev.dtype}")
    print(f"{'Y_dev':<10} {str(tuple(Y_dev.shape)):<25} {str(Y_dev.device):<10} {Y_dev.dtype}")
    print(f"{'X_test':<10} {str(tuple(X_test.shape)):<25} {str(X_test.device):<10} {X_test.dtype}")
    print(f"{'Y_test':<10} {str(tuple(Y_test.shape)):<25} {str(Y_test.device):<10} {Y_test.dtype}")
    
    return X_train, Y_train, X_dev, Y_dev, X_test, Y_test


# ============================================================================
# EXÉCUTION
# ============================================================================

X_train, Y_train, X_dev, Y_dev, X_test, Y_test = load_mnist_data_cnn(
    DATA_TRAIN_PATH, 
    DATA_TEST_PATH,
    train_split=0.8,
    shuffle=True
)


# 🏗️ CELLULE 3 : Architecture CNN

In [ ]:
"""
╔══════════════════════════════════════════════════════════════════════════╗
║                      DÉFINITION DU MODÈLE CNN                            ║
╚══════════════════════════════════════════════════════════════════════════╝

Architecture flexible :
- Nombre variable de couches convolutionnelles
- Nombre variable de couches fully connected
- Support de Batch Normalization, Dropout, et différentes activations
"""

class CNN(nn.Module):
    """
    Réseau de Neurones Convolutionnel (CNN) flexible pour MNIST.
    
    Architecture :
    ┌─────────────────────────────────────────────────────────────┐
    │  INPUT (1, 28, 28)                                          │
    │     ↓                                                       │
    │  [Conv2D → (BatchNorm) → Activation → MaxPool → Dropout]    │  × n couches
    │     ↓                                                       │
    │  Flatten                                                    │
    │     ↓                                                       │
    │  [Linear → (BatchNorm) → Activation → Dropout]              │  × n couches
    │     ↓                                                       │
    │  Linear(10) → OUTPUT (logits)                               │
    └─────────────────────────────────────────────────────────────┘
    
    Args:
        conv_channels (list): Nombre de filtres par couche conv. Ex: [32, 64, 128]
        fc_sizes (list): Taille des couches fully connected. Ex: [128, 64]
        kernel_size (int): Taille des kernels convolutionnels (3 ou 5)
        pool_size (int): Taille du max pooling (typiquement 2)
        dropout (float): Taux de dropout (0.0 à 1.0)
        batch_norm (bool): Utiliser Batch Normalization
        activation (str): Type d'activation ('relu', 'leaky_relu', 'gelu')
    
    Example:
        >>> model = CNN(
        ...     conv_channels=[64, 64],
        ...     fc_sizes=[160],
        ...     kernel_size=5,
        ...     dropout=0.5,
        ...     batch_norm=False,
        ...     activation='gelu'
        ... )
        >>> print(model)
    """
    
    def __init__(self, conv_channels=[32, 64], fc_sizes=[128],
                 kernel_size=3, pool_size=2, dropout=0.5,
                 batch_norm=True, activation='relu'):
        
        super(CNN, self).__init__()
        
        # ====================================================================
        # CONFIGURATION
        # ====================================================================
        
        self.use_batch_norm = batch_norm
        self.conv_layers = nn.ModuleList()
        self.conv_batch_norms = nn.ModuleList() if batch_norm else None
        
        # ====================================================================
        # FONCTION D'ACTIVATION
        # ====================================================================
        
        activation_functions = {
            'relu': nn.ReLU(),
            'leaky_relu': nn.LeakyReLU(0.1),
            'gelu': nn.GELU()
        }
        self.activation = activation_functions.get(activation, nn.ReLU())
        
        # ====================================================================
        # COUCHES CONVOLUTIONNELLES
        # ====================================================================
        
        in_channels = 1  # MNIST = 1 canal (grayscale)
        
        for out_channels in conv_channels:
            # Conv2D avec padding=1 pour préserver la taille
            self.conv_layers.append(
                nn.Conv2d(in_channels, out_channels, 
                         kernel_size=kernel_size, padding=1)
            )
            
            if batch_norm:
                self.conv_batch_norms.append(nn.BatchNorm2d(out_channels))
            
            in_channels = out_channels
        
        # Pooling et Dropout
        self.pool = nn.MaxPool2d(pool_size, pool_size)
        self.dropout_conv = nn.Dropout2d(dropout * 0.5)  # Dropout réduit pour conv
        self.dropout_fc = nn.Dropout(dropout)
        
        # ====================================================================
        # CALCUL AUTOMATIQUE DE LA TAILLE APRÈS CONVOLUTIONS
        # ====================================================================
        
        flatten_size = self._get_flatten_size(conv_channels, kernel_size, pool_size)
        
        # ====================================================================
        # COUCHES FULLY CONNECTED
        # ====================================================================
        
        self.fc_layers = nn.ModuleList()
        self.fc_batch_norms = nn.ModuleList() if batch_norm else None
        
        in_features = flatten_size
        for fc_size in fc_sizes:
            self.fc_layers.append(nn.Linear(in_features, fc_size))
            
            if batch_norm:
                self.fc_batch_norms.append(nn.BatchNorm1d(fc_size))
            
            in_features = fc_size
        
        # Couche de sortie (10 classes)
        self.output_layer = nn.Linear(in_features, 10)
        
        # ====================================================================
        # INITIALISATION DES POIDS
        # ====================================================================
        
        self._initialize_weights()
    
    
    def _get_flatten_size(self, conv_channels, kernel_size, pool_size):
        """
        Calcule automatiquement la taille du vecteur après convolutions.
        
        Utilise un forward pass "dummy" avec une image fictive pour déterminer
        la dimension exacte du vecteur aplati.
        
        Returns:
            int: Nombre d'éléments dans le vecteur aplati
        """
        with torch.no_grad():
            # Créer une image fictive (batch=1, channels=1, H=28, W=28)
            x = torch.zeros(1, 1, 28, 28)
            
            # Passer à travers les conv layers
            for i in range(len(conv_channels)):
                x = self.conv_layers[i](x)
                if self.use_batch_norm:
                    x = self.conv_batch_norms[i](x)
                x = self.activation(x)
                x = self.pool(x)
            
            # Retourner le nombre total d'éléments
            return int(np.prod(x.size()[1:]))
    
    
    def _initialize_weights(self):
        """
        Initialise les poids avec la méthode de Kaiming He (pour ReLU).
        
        Améliore la convergence en évitant les gradients qui explosent/disparaissent.
        """
        for m in self.modules():
            if isinstance(m, (nn.Conv2d, nn.Linear)):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
                if m.bias is not None:
                    nn.init.zeros_(m.bias)
    
    
    def forward(self, x):
        """
        Forward pass du CNN.
        
        Args:
            x (Tensor): Images de shape (batch_size, 1, 28, 28)
        
        Returns:
            Tensor: Logits de shape (batch_size, 10)
        """
        
        # ====================================================================
        # BLOC CONVOLUTIONNEL
        # ====================================================================
        
        for i, conv in enumerate(self.conv_layers):
            x = conv(x)
            
            if self.use_batch_norm:
                x = self.conv_batch_norms[i](x)
            
            x = self.activation(x)
            x = self.pool(x)
            x = self.dropout_conv(x)
        
        # ====================================================================
        # APLATIR
        # ====================================================================
        
        x = x.view(x.size(0), -1)  # (batch, channels, H, W) → (batch, flatten_size)
        
        # ====================================================================
        # BLOC FULLY CONNECTED
        # ====================================================================
        
        for i, fc in enumerate(self.fc_layers):
            x = fc(x)
            
            if self.use_batch_norm:
                x = self.fc_batch_norms[i](x)
            
            x = self.activation(x)
            x = self.dropout_fc(x)
        
        # ====================================================================
        # SORTIE
        # ====================================================================
        
        x = self.output_layer(x)
        return x


# ============================================================================
# FONCTIONS UTILITAIRES
# ============================================================================

def evaluate_model(model, X, Y, batch_size=256):
    """
    Évalue l'accuracy du modèle sur un dataset.
    
    Effectue l'évaluation par batches pour gérer les grands datasets.
    
    Args:
        model (nn.Module): Modèle à évaluer
        X (Tensor): Images
        Y (Tensor): Labels
        batch_size (int): Taille des batches
    
    Returns:
        float: Accuracy (entre 0 et 1)
    """
    model.eval()
    correct = 0
    total = 0
    
    with torch.no_grad():
        for i in range(0, len(X), batch_size):
            batch_X = X[i:i+batch_size]
            batch_Y = Y[i:i+batch_size]
            
            outputs = model(batch_X)
            _, predicted = torch.max(outputs, 1)
            correct += (predicted == batch_Y).sum().item()
            total += batch_Y.size(0)
    
    return correct / total


print("✅ Architecture CNN définie avec calcul automatique des dimensions")


# 🔍 CELLULE 4 : Optimisation Bayésienne des Hyperparamètres


In [ ]:
"""
╔══════════════════════════════════════════════════════════════════════════╗
║              OPTIMISATION BAYÉSIENNE AVEC OPTUNA                         ║
╚══════════════════════════════════════════════════════════════════════════╝

Objectif : Trouver automatiquement les meilleurs hyperparamètres du CNN

Hyperparamètres optimisés :
- Architecture : nombre et taille des couches conv/fc
- Kernel size : 3×3 ou 5×5
- Dropout : taux de régularisation
- Learning rate : vitesse d'apprentissage
- L2 regularization : pénalité sur les poids
- Batch normalization : on/off
- Fonction d'activation : ReLU/LeakyReLU/GELU
"""

print("\n" + "="*70)
print("🔬 OPTIMISATION BAYÉSIENNE CNN")
print("="*70)

# ============================================================================
# FONCTION D'ENTRAÎNEMENT POUR OPTUNA
# ============================================================================

def train_cnn(X, Y, model, optimizer, criterion, epochs=10, batch_size=128, grad_clip=None):
    """
    Entraîne le CNN par mini-batches avec gradient clipping optionnel.
    
    Args:
        X (Tensor): Images d'entraînement (N, 1, 28, 28)
        Y (Tensor): Labels (N,)
        model (nn.Module): Modèle CNN à entraîner
        optimizer (torch.optim): Optimiseur (AdamW, SGD, etc.)
        criterion (nn.Module): Fonction de perte (CrossEntropyLoss)
        epochs (int): Nombre d'epochs
        batch_size (int): Taille des batches
        grad_clip (float|None): Valeur max pour gradient clipping
    
    Returns:
        nn.Module: Modèle entraîné
    
    Notes:
        - Shuffle automatique des données à chaque epoch
        - Gradient clipping pour stabiliser l'entraînement
    """
    model.train()
    
    for epoch in range(epochs):
        # Mélanger les indices à chaque epoch
        indices = torch.randperm(X.size(0))
        
        # Entraînement par batches
        for i in range(0, X.size(0), batch_size):
            batch_indices = indices[i:i+batch_size]
            batch_X = X[batch_indices]
            batch_Y = Y[batch_indices]
            
            # Forward pass
            optimizer.zero_grad()
            outputs = model(batch_X)
            loss = criterion(outputs, batch_Y)
            
            # Backward pass
            loss.backward()
            
            # Gradient clipping (évite l'explosion des gradients)
            if grad_clip:
                torch.nn.utils.clip_grad_norm_(model.parameters(), grad_clip)
            
            # Mise à jour des poids
            optimizer.step()
    
    return model


# ============================================================================
# FONCTION OBJECTIF POUR OPTUNA
# ============================================================================

def cnn_objective(trial):
    """
    Fonction objectif pour Optuna : teste une configuration d'hyperparamètres.
    
    Optuna va appeler cette fonction plusieurs fois avec différentes combinaisons
    d'hyperparamètres pour trouver la meilleure configuration.
    
    Args:
        trial (optuna.Trial): Object trial contenant les suggestions d'hyperparamètres
    
    Returns:
        float: Accuracy sur le dev set (métrique à maximiser)
    
    Raises:
        optuna.TrialPruned: Si l'architecture est invalide ou les résultats médiocres
    """
    
    # ========================================================================
    # ÉTAPE 1 : SUGGÉRER LES HYPERPARAMÈTRES
    # ========================================================================
    
    # Architecture convolutionnelle
    n_conv_layers = trial.suggest_int('n_conv_layers', 2, 3)
    conv_channels = [
        trial.suggest_categorical(f'conv_ch_{i}', [32, 64, 128, 256])
        for i in range(n_conv_layers)
    ]
    
    # Architecture fully connected
    n_fc_layers = trial.suggest_int('n_fc_layers', 1, 2)
    fc_sizes = [
        trial.suggest_int(f'fc_size_{i}', 64, 256, step=32)
        for i in range(n_fc_layers)
    ]
    
    # Hyperparamètres d'entraînement
    kernel_size = trial.suggest_categorical('kernel_size', [3, 5])
    dropout = trial.suggest_float('dropout', 0.3, 0.6)
    lr = trial.suggest_float('learning_rate', 1e-4, 1e-2, log=True)
    l2_lambda = trial.suggest_float('l2_lambda', 1e-5, 1e-3, log=True)
    batch_norm = trial.suggest_categorical('batch_norm', [True, False])
    activation = trial.suggest_categorical('activation', ['relu', 'leaky_relu', 'gelu'])
    
    # ========================================================================
    # ÉTAPE 2 : VALIDATION DE L'ARCHITECTURE
    # ========================================================================
    
    # Calculer la taille de l'image après n_conv_layers
    # Chaque MaxPool2d(2, 2) divise par 2 : 28 → 14 → 7 → 3
    image_size = 28
    for _ in range(n_conv_layers):
        image_size = image_size // 2
    
    # Vérifier que le kernel n'est pas trop grand
    if kernel_size == 5 and image_size < 5:
        raise optuna.TrialPruned(
            f"⚠️  Kernel {kernel_size}×{kernel_size} trop grand pour image {image_size}×{image_size}"
        )
    
    # ========================================================================
    # ÉTAPE 3 : CRÉER LE MODÈLE
    # ========================================================================
    
    try:
        model = CNN(
            conv_channels=conv_channels,
            fc_sizes=fc_sizes,
            kernel_size=kernel_size,
            dropout=dropout,
            batch_norm=batch_norm,
            activation=activation
        ).to(device)
    except RuntimeError as e:
        # Pruner si erreur de dimension
        raise optuna.TrialPruned(f"❌ Erreur architecture : {e}")
    
    # ========================================================================
    # ÉTAPE 4 : ENTRAÎNER LE MODÈLE
    # ========================================================================
    
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.AdamW(model.parameters(), lr=lr, weight_decay=l2_lambda)
    
    model = train_cnn(
        X_train, Y_train, model, optimizer, criterion,
        epochs=15, batch_size=128, grad_clip=1.0
    )
    
    # ========================================================================
    # ÉTAPE 5 : ÉVALUER SUR DEV SET
    # ========================================================================
    
    accuracy = evaluate_model(model, X_dev, Y_dev)
    
    return accuracy


# ============================================================================
# LANCER L'OPTIMISATION
# ============================================================================

print("\n🚀 Lancement de l'optimisation (50 trials, timeout 1h)...")
print("   Chaque trial teste une configuration d'hyperparamètres différente\n")

# Créer l'étude Optuna
study_cnn = optuna.create_study(
    direction='maximize',  # Maximiser l'accuracy
    sampler=optuna.samplers.TPESampler(seed=RANDOM_SEED),  # Algorithme bayésien
    pruner=optuna.pruners.MedianPruner()  # Arrête les trials peu prometteurs
)

# Optimiser
study_cnn.optimize(
    cnn_objective, 
    n_trials=50,  # Nombre de configurations à tester
    timeout=3600,  # Arrêt après 1h max
    show_progress_bar=True
)

# ============================================================================
# RÉSULTATS DE L'OPTIMISATION
# ============================================================================

print("\n" + "="*70)
print("🏆 RÉSULTATS DE L'OPTIMISATION")
print("="*70)

print(f"\n✅ Meilleure accuracy : {study_cnn.best_value:.4f}")
print(f"\n📋 Meilleurs hyperparamètres :")
for key, value in study_cnn.best_params.items():
    print(f"   • {key:<20} : {value}")

# ============================================================================
# SAUVEGARDER LES HYPERPARAMÈTRES
# ============================================================================

# Reconstruire le dictionnaire complet
best_params = study_cnn.best_params
n_conv = best_params['n_conv_layers']
n_fc = best_params['n_fc_layers']

hyperparams_to_save = {
    'n_conv_layers': n_conv,
    'conv_channels': [best_params[f'conv_ch_{i}'] for i in range(n_conv)],
    'n_fc_layers': n_fc,
    'fc_sizes': [best_params[f'fc_size_{i}'] for i in range(n_fc)],
    'kernel_size': best_params['kernel_size'],
    'dropout': best_params['dropout'],
    'learning_rate': best_params['learning_rate'],
    'l2_lambda': best_params['l2_lambda'],
    'batch_norm': best_params['batch_norm'],
    'activation': best_params['activation'],
    'best_dev_accuracy': study_cnn.best_value
}

with open(HYPERPARAMS_PATH, 'w') as f:
    json.dump(hyperparams_to_save, f, indent=2)

print(f"\n💾 Hyperparamètres sauvegardés dans : {HYPERPARAMS_PATH}")

# ============================================================================
# VISUALISATION DES RÉSULTATS D'OPTIMISATION
# ============================================================================

# Graphique 1 : Historique d'optimisation
fig1 = plot_optimization_history(study_cnn)
fig1.update_layout(
    title="Historique d'Optimisation Bayésienne",
    xaxis_title="Trial",
    yaxis_title="Dev Accuracy"
)
fig1.show()

# Graphique 2 : Importance des hyperparamètres
fig2 = plot_param_importances(study_cnn)
fig2.update_layout(
    title="Importance des Hyperparamètres"
)
fig2.show()

print("\n" + "="*70)
print("✅ OPTIMISATION TERMINÉE")
print("="*70)


# 🎯 CELLULE 5 : Entraînement Final avec Early Stopping

In [ ]:
"""
╔══════════════════════════════════════════════════════════════════════════╗
║                  ENTRAÎNEMENT FINAL DU MODÈLE                            ║
╚══════════════════════════════════════════════════════════════════════════╝

Utilise les meilleurs hyperparamètres trouvés par Optuna
Implémente l'early stopping pour éviter le surapprentissage
"""

# ============================================================================
# CHARGER LES MEILLEURS HYPERPARAMÈTRES
# ============================================================================

with open(HYPERPARAMS_PATH, 'r') as f:
    best_params = json.load(f)

print("="*70)
print("⚙️  HYPERPARAMÈTRES CHARGÉS")
print("="*70)
print(json.dumps(best_params, indent=2))

# ============================================================================
# FONCTION D'ENTRAÎNEMENT AVEC EARLY STOPPING
# ============================================================================

def train_cnn_with_early_stopping(X_train, Y_train, X_val, Y_val, hyperparams,
                                   patience=5, max_epochs=100, batch_size=128):
    """
    Entraîne le CNN avec early stopping et learning rate scheduling.
    
    Early Stopping : Arrête l'entraînement si la val accuracy ne s'améliore pas
    pendant 'patience' évaluations consécutives.
    
    Args:
        X_train, Y_train (Tensor): Données d'entraînement
        X_val, Y_val (Tensor): Données de validation
        hyperparams (dict): Hyperparamètres du modèle
        patience (int): Nombre d'epochs sans amélioration avant arrêt
        max_epochs (int): Nombre maximum d'epochs
        batch_size (int): Taille des batches
    
    Returns:
        tuple: (model_final, best_val_acc, history)
            - model_final : Modèle avec les meilleurs poids
            - best_val_acc : Meilleure accuracy sur validation
            - history : Dictionnaire avec historique d'entraînement
    """
    
    # ========================================================================
    # INITIALISATION DU MODÈLE
    # ========================================================================
    
    model = CNN(
        conv_channels=hyperparams['conv_channels'],
        fc_sizes=hyperparams['fc_sizes'],
        kernel_size=hyperparams['kernel_size'],
        dropout=hyperparams['dropout'],
        batch_norm=hyperparams['batch_norm'],
        activation=hyperparams['activation']
    ).to(device)
    
    # Critère de perte et optimiseur
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.AdamW(
        model.parameters(),
        lr=hyperparams['learning_rate'],
        weight_decay=hyperparams['l2_lambda']
    )
    
    # Learning rate scheduler : réduit le LR si plateau
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(
        optimizer, mode='max', patience=5, factor=0.5, verbose=True
    )
    
    # ========================================================================
    # INITIALISATION DE L'HISTORIQUE
    # ========================================================================
    
    history = {
        'epochs': [],
        'train_losses': [],
        'val_losses': [],
        'train_accs': [],
        'val_accs': []
    }
    
    best_val_acc = 0.0
    patience_counter = 0
    best_model_state = None
    
    # ========================================================================
    # BOUCLE D'ENTRAÎNEMENT
    # ========================================================================
    
    print("\n" + "="*70)
    print("🚀 DÉMARRAGE DE L'ENTRAÎNEMENT")
    print("="*70)
    
    for epoch in range(max_epochs):
        model.train()
        
        # Shuffle des données
        indices = torch.randperm(X_train.size(0))
        epoch_loss = 0.0
        
        # Entraînement par batches
        for i in range(0, X_train.size(0), batch_size):
            batch_indices = indices[i:i+batch_size]
            batch_X = X_train[batch_indices]
            batch_Y = Y_train[batch_indices]
            
            # Forward + backward
            optimizer.zero_grad()
            outputs = model(batch_X)
            loss = criterion(outputs, batch_Y)
            loss.backward()
            
            # Gradient clipping
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
            
            optimizer.step()
            epoch_loss += loss.item()
        
        # ====================================================================
        # VALIDATION (tous les 2 epochs)
        # ====================================================================
        
        if epoch % 2 == 0:
            model.eval()
            
            with torch.no_grad():
                # Évaluer train accuracy
                train_acc = evaluate_model(model, X_train, Y_train, batch_size)
                train_loss = epoch_loss / (X_train.size(0) // batch_size)
                
                # Évaluer validation
                val_outputs = model(X_val)
                val_loss = criterion(val_outputs, Y_val).item()
                val_acc = evaluate_model(model, X_val, Y_val, batch_size)
            
            # Learning rate scheduling
            scheduler.step(val_acc)
            
            # Sauvegarder l'historique
            history['epochs'].append(epoch)
            history['train_losses'].append(train_loss)
            history['val_losses'].append(val_loss)
            history['train_accs'].append(train_acc)
            history['val_accs'].append(val_acc)
            
            # Early stopping check
            if val_acc > best_val_acc:
                best_val_acc = val_acc
                best_model_state = model.state_dict().copy()
                patience_counter = 0
            else:
                patience_counter += 1
            
            # Affichage périodique
            if epoch % 10 == 0:
                print(f"Epoch {epoch:3d} | "
                      f"Train: Acc={train_acc:.4f} Loss={train_loss:.4f} | "
                      f"Val: Acc={val_acc:.4f} Loss={val_loss:.4f}")
            
            # Arrêt si pas d'amélioration
            if patience_counter >= patience:
                print(f"✓ Early stopping à l'epoch {epoch}")
                break
    
    # ========================================================================
    # CHARGER LES MEILLEURS POIDS
    # ========================================================================
    
    if best_model_state:
        model.load_state_dict(best_model_state)
    
    # ========================================================================
    # VISUALISATION DE L'ENTRAÎNEMENT
    # ========================================================================
    
    fig, ax1 = plt.subplots(figsize=(14, 7))
    
    # Axe 1 : Accuracy
    ax1.plot(history['epochs'], history['train_accs'], 
             label='Train Accuracy', linewidth=2.5, color='#4ECDC4', marker='o')
    ax1.plot(history['epochs'], history['val_accs'], 
             label='Val Accuracy', linewidth=2.5, linestyle='--', 
             color='#45B7D1', marker='s')
    ax1.set_xlabel('Epochs', fontsize=13, fontweight='bold')
    ax1.set_ylabel('Accuracy', color='tab:blue', fontsize=13, fontweight='bold')
    ax1.tick_params(axis='y', labelcolor='tab:blue')
    ax1.legend(loc='lower right', fontsize=11)
    ax1.set_ylim(0, 1)
    ax1.grid(True, alpha=0.3, linestyle='--')
    
    # Axe 2 : Loss
    ax2 = ax1.twinx()
    ax2.plot(history['epochs'], history['train_losses'], 
             label='Train Loss', linewidth=2.5, linestyle='-', 
             color='#FF6B6B', marker='o', alpha=0.7)
    ax2.plot(history['epochs'], history['val_losses'], 
             label='Val Loss', linewidth=2.5, linestyle='--', 
             color='#EE5A6F', marker='s', alpha=0.7)
    ax2.set_ylabel('Loss', color='tab:red', fontsize=13, fontweight='bold')
    ax2.tick_params(axis='y', labelcolor='tab:red')
    ax2.legend(loc='upper right', fontsize=11)
    
    max_loss = max(max(history['train_losses']), max(history['val_losses']))
    ax2.set_ylim(0, min(max_loss * 1.1, 3.0))
    
    plt.title('CNN Training History', fontsize=16, pad=20, fontweight='bold')
    plt.tight_layout()
    plt.savefig(GRAPH_DIR / 'cnn_training_history.png', dpi=300, bbox_inches='tight')
    plt.show()
    
    return model, best_val_acc, history


# ============================================================================
# ENTRAÎNER LE MODÈLE FINAL
# ============================================================================

print("\n" + "="*70)
print("🎯 ENTRAÎNEMENT FINAL DU CNN")
print("="*70)

model_final, final_val_acc, history = train_cnn_with_early_stopping(
    X_train, Y_train, X_dev, Y_dev, best_params, 
    patience=5, max_epochs=100, batch_size=128
)

# ============================================================================
# SAUVEGARDER LE MODÈLE
# ============================================================================

torch.save({
    'model_state_dict': model_final.state_dict(),
    'hyperparams': best_params,
    'val_accuracy': final_val_acc,
    'train_history': history
}, MODEL_PATH)

print(f"\n" + "="*70)
print(f"✅ MODÈLE CNN SAUVEGARDÉ")
print("="*70)
print(f"📁 Fichier : {MODEL_PATH}")
print(f"🎯 Val Accuracy : {final_val_acc:.4f}")
print(f"📊 Historique : {len(history['epochs'])} epochs enregistrés")


# 📊 CELLULE 6 : Évaluation sur le Test Set

In [ ]:
"""
╔══════════════════════════════════════════════════════════════════════════╗
║                    ÉVALUATION SUR LE TEST SET                            ║
╚══════════════════════════════════════════════════════════════════════════╝

Évaluation finale du modèle sur des données jamais vues pendant l'entraînement
Métriques : Accuracy, AUC-ROC, Confusion Matrix, Classification Report
"""

print("\n" + "="*70)
print("📊 ÉVALUATION CNN SUR TEST SET")
print("="*70)

# ============================================================================
# CHARGER LE MODÈLE SAUVEGARDÉ
# ============================================================================

print("\n🔄 Chargement du modèle...")

checkpoint = torch.load(MODEL_PATH, map_location=device)
best_params = checkpoint['hyperparams']

model_final = CNN(
    conv_channels=best_params['conv_channels'],
    fc_sizes=best_params['fc_sizes'],
    kernel_size=best_params['kernel_size'],
    dropout=best_params['dropout'],
    batch_norm=best_params['batch_norm'],
    activation=best_params['activation']
).to(device)

model_final.load_state_dict(checkpoint['model_state_dict'])

print(f"✅ Modèle chargé")
print(f"   Architecture Conv : {best_params['conv_channels']}")
print(f"   Architecture FC : {best_params['fc_sizes']}")
print(f"   Val Accuracy : {checkpoint['val_accuracy']:.4f}")

# ============================================================================
# PRÉDICTIONS SUR LE TEST SET
# ============================================================================

print("\n🔄 Génération des prédictions...")

model_final.eval()
with torch.no_grad():
    test_outputs = model_final(X_test)
    test_probs = torch.softmax(test_outputs, dim=1)
    _, test_pred = torch.max(test_outputs, 1)
    
    # Convertir en numpy pour les métriques sklearn
    y_true = Y_test.cpu().numpy()
    y_pred = test_pred.cpu().numpy()
    y_probs = test_probs.cpu().numpy()
    
    # Calculer les métriques
    test_acc = (test_pred == Y_test).float().mean().item()
    auc_score = roc_auc_score(y_true, y_probs, multi_class='ovr', average='macro')
    criterion = nn.CrossEntropyLoss()
    test_loss = criterion(test_outputs, Y_test).item()

# ============================================================================
# AFFICHAGE DES RÉSULTATS GLOBAUX
# ============================================================================

print("\n" + "="*70)
print("🏆 RÉSULTATS SUR LE TEST SET")
print("="*70)
print(f"\n{'Métrique':<25} {'Valeur':<15} {'Interprétation'}")
print("-" * 70)
print(f"{'Test Accuracy':<25} {test_acc:<15.4f} {'🥇 Excellent' if test_acc > 0.99 else '🥈 Très bon' if test_acc > 0.98 else '🥉 Bon'}")
print(f"{'AUC-ROC (macro)':<25} {auc_score:<15.4f} {'🥇 Excellent' if auc_score > 0.99 else '🥈 Très bon' if auc_score > 0.98 else '🥉 Bon'}")
print(f"{'Test Loss':<25} {test_loss:<15.4f} {'✅ Faible' if test_loss < 0.1 else '⚠️  Modéré'}")

# ============================================================================
# CLASSIFICATION REPORT DÉTAILLÉ
# ============================================================================

print("\n" + "="*70)
print("📋 CLASSIFICATION REPORT (par classe)")
print("="*70)
print(classification_report(y_true, y_pred, digits=4, target_names=[str(i) for i in range(10)]))

# ============================================================================
# MATRICE DE CONFUSION
# ============================================================================

print("\n📊 Génération de la matrice de confusion...")

cm = confusion_matrix(y_true, y_pred)

fig, ax = plt.subplots(figsize=(12, 10))
im = ax.imshow(cm, cmap='Blues', interpolation='nearest')

# Titre et colorbar
plt.title('Matrice de Confusion - CNN Test Set', fontsize=16, pad=20, fontweight='bold')
cbar = plt.colorbar(im, ax=ax)
cbar.set_label('Nombre de prédictions', fontsize=12)

# Ticks
tick_marks = np.arange(10)
plt.xticks(tick_marks, range(10), fontsize=11)
plt.yticks(tick_marks, range(10), fontsize=11)
plt.xlabel('Prédiction', fontsize=13, fontweight='bold')
plt.ylabel('Vérité Terrain', fontsize=13, fontweight='bold')

# Annotations
thresh = cm.max() / 2.

for i in range(10):
    for j in range(10):
        value = cm[i, j]
        
        # Surligner en rouge les erreurs importantes (> 10)
        if i != j and value > 10:
            rect = mpatches.Rectangle(
                (j-0.4, i-0.4), 0.8, 0.8,
                fill=True, facecolor='red',
                alpha=0.25, edgecolor='darkred', linewidth=2.5
            )
            ax.add_patch(rect)
        
        # Couleur du texte (blanc si fond sombre, noir sinon)
        text_color = 'white' if value > thresh else 'black'
        font_weight = 'bold' if (i != j and value > 10) else 'normal'
        
        plt.text(j, i, value, ha='center', va='center',
                color=text_color, fontsize=11, weight=font_weight)

plt.tight_layout()
plt.savefig(GRAPH_DIR / 'cnn_confusion_matrix.png', dpi=300, bbox_inches='tight')
plt.show()

# ============================================================================
# ACCURACY PAR CLASSE
# ============================================================================

print("\n" + "="*70)
print("📊 ACCURACY PAR CLASSE")
print("="*70)

class_accs = []
print(f"\n{'Chiffre':<10} {'Accuracy':<12} {'Nb Exemples':<15} {'Qualité'}")
print("-" * 60)

for digit in range(10):
    mask = (Y_test == digit)
    if mask.sum() > 0:
        digit_acc = (test_pred[mask] == Y_test[mask]).float().mean().item()
        n_examples = mask.sum().item()
        class_accs.append(digit_acc)
        
        quality = "🥇" if digit_acc > 0.99 else "🥈" if digit_acc > 0.98 else "🥉" if digit_acc > 0.97 else "⚠️"
        
        print(f"{digit:<10} {digit_acc:<12.4f} {n_examples:<15} {quality}")

print("-" * 60)
print(f"{'Moyenne':<10} {np.mean(class_accs):<12.4f}")
print(f"{'Écart-type':<10} {np.std(class_accs):<12.4f}")

# ============================================================================
# IDENTIFIER LES CLASSES PROBLÉMATIQUES
# ============================================================================

print("\n" + "="*70)
print("🔍 ANALYSE DES CONFUSIONS")
print("="*70)

# Trouver les paires de chiffres les plus confondues
confusions = []
for i in range(10):
    for j in range(10):
        if i != j and cm[i, j] > 5:  # Au moins 5 erreurs
            confusions.append((i, j, cm[i, j]))

confusions.sort(key=lambda x: x[2], reverse=True)

print(f"\n🔴 Top 5 des confusions (Vrai → Prédit) :")
for i, (true_class, pred_class, count) in enumerate(confusions[:5], 1):
    percentage = (count / cm[true_class].sum()) * 100
    print(f"   {i}. Chiffre {true_class} → {pred_class} : {count} erreurs ({percentage:.2f}%)")

print("\n" + "="*70)
print("✅ ÉVALUATION TERMINÉE")
print("="*70)


# 🎨 CELLULE 7 : Visualisation des Filtres et Feature Maps

In [ ]:
"""
╔══════════════════════════════════════════════════════════════════════════╗
║              VISUALISATION DES FILTRES CONVOLUTIONNELS                   ║
╚══════════════════════════════════════════════════════════════════════════╝

Objectif : Comprendre ce que le CNN a appris à détecter

1. Filtres (kernels) : Motifs que le CNN recherche dans les images
2. Feature Maps : Activations produites par ces filtres sur une image
"""

print("\n" + "="*70)
print("🎨 VISUALISATION DES FILTRES CNN")
print("="*70)

# ============================================================================
# EXTRACTION DES FILTRES DE LA PREMIÈRE COUCHE
# ============================================================================

print("\n🔄 Extraction des filtres convolutionnels...")

# Extraire les poids de la première couche conv
first_conv = model_final.conv_layers[0]
filters = first_conv.weight.data.cpu().numpy()

print(f"✅ Filtres extraits")
print(f"   Shape : {filters.shape}")
print(f"   Nombre de filtres : {filters.shape[0]}")
print(f"   Taille des filtres : {filters.shape[2]}×{filters.shape[3]}")

# ============================================================================
# VISUALISATION DES FILTRES
# ============================================================================

print("\n📊 Génération de la visualisation des filtres...")

n_filters = min(32, filters.shape[0])
n_cols = 8
n_rows = (n_filters + n_cols - 1) // n_cols  # Arrondi supérieur

fig, axes = plt.subplots(n_rows, n_cols, figsize=(18, 2.5 * n_rows))
fig.suptitle('Filtres Convolutionnels - Première Couche\n'
             'Ces filtres détectent des patterns de base (bords, coins, textures)', 
             fontsize=16, y=0.98, fontweight='bold')

for i, ax in enumerate(axes.flat):
    if i < n_filters:
        # Extraire le filtre (canal 0 car images grayscale)
        filt = filters[i, 0, :, :]
        
        # Normaliser pour visualisation [0, 1]
        filt_norm = (filt - filt.min()) / (filt.max() - filt.min() + 1e-8)
        
        # Afficher
        im = ax.imshow(filt_norm, cmap='viridis', interpolation='nearest')
        ax.set_title(f'Filtre {i}', fontsize=10, fontweight='bold')
        
        # Colorbar pour chaque filtre
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    else:
        ax.axis('off')
    
    ax.set_xticks([])
    ax.set_yticks([])

plt.tight_layout()
plt.savefig(GRAPH_DIR / 'cnn_conv_filters.png', dpi=300, bbox_inches='tight')
plt.show()

# ============================================================================
# ANALYSE STATISTIQUE DES FILTRES
# ============================================================================

print("\n📊 Statistiques des filtres :")
print(f"   Min : {filters.min():.4f}")
print(f"   Max : {filters.max():.4f}")
print(f"   Moyenne : {filters.mean():.4f}")
print(f"   Écart-type : {filters.std():.4f}")

# ============================================================================
# VISUALISATION DES FEATURE MAPS
# ============================================================================

print("\n" + "="*70)
print("🔬 VISUALISATION DES FEATURE MAPS")
print("="*70)

print("\n🔄 Génération des feature maps sur un exemple du test set...")

# ============================================================================
# SÉLECTIONNER UN EXEMPLE
# ============================================================================

sample_idx = 0
sample_image = X_test[sample_idx:sample_idx+1]
sample_label = Y_test[sample_idx].item()

# ============================================================================
# CAPTURER LES ACTIVATIONS AVEC DES HOOKS
# ============================================================================

# Dictionnaire pour stocker les activations
activations = {}

def get_activation(name):
    """
    Factory function pour créer des hooks.
    
    Un hook est une fonction callback qui s'exécute automatiquement
    après le forward pass d'une couche.
    
    Args:
        name (str): Nom de la couche pour identifier l'activation
    
    Returns:
        function: Hook function
    """
    def hook(model, input, output):
        activations[name] = output.detach()
    return hook

# Enregistrer hooks sur toutes les couches convolutionnelles
hooks = []
for i, layer in enumerate(model_final.conv_layers):
    hooks.append(layer.register_forward_hook(get_activation(f'conv{i}')))

# Forward pass pour déclencher les hooks
model_final.eval()
with torch.no_grad():
    output = model_final(sample_image)
    pred = output.argmax().item()

# Retirer les hooks (important pour éviter les fuites mémoire)
for hook in hooks:
    hook.remove()

print(f"✅ Activations capturées pour {len(activations)} couches")

# ============================================================================
# VISUALISER LES FEATURE MAPS DE LA PREMIÈRE COUCHE
# ============================================================================

print("\n📊 Visualisation des feature maps (Conv Layer 0)...")

feature_maps = activations['conv0'].cpu().numpy()[0]  # Shape: (n_channels, H, W)

n_maps = min(16, feature_maps.shape[0])
fig, axes = plt.subplots(4, 4, figsize=(14, 14))
fig.suptitle(f'Feature Maps - Couche Convolutionnelle 0\n'
             f'Image : Chiffre {sample_label} | Prédiction : {pred} | '
             f'{"✅ Correct" if pred == sample_label else "❌ Erreur"}',
             fontsize=15, y=0.98, fontweight='bold')

for i, ax in enumerate(axes.flat):
    if i < n_maps:
        # Extraire la feature map
        fmap = feature_maps[i]
        
        # Afficher
        im = ax.imshow(fmap, cmap='viridis', interpolation='bilinear')
        ax.set_title(f'Channel {i}', fontsize=11, fontweight='bold')
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    else:
        ax.axis('off')
    
    ax.set_xticks([])
    ax.set_yticks([])

plt.tight_layout()
plt.savefig(GRAPH_DIR / 'cnn_feature_maps.png', dpi=300, bbox_inches='tight')
plt.show()

# ============================================================================
# AFFICHER L'IMAGE ORIGINALE
# ============================================================================

print("\n📸 Image originale :")

fig, ax = plt.subplots(figsize=(5, 5))
im = ax.imshow(sample_image.cpu().squeeze(), cmap='gray')
ax.set_title(f'Image Originale\n'
             f'Vrai Label : {sample_label} | Prédiction : {pred}',
             fontsize=13, fontweight='bold', pad=15)
ax.axis('off')
plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
plt.tight_layout()
plt.savefig(GRAPH_DIR / 'cnn_sample_image.png', dpi=300, bbox_inches='tight')
plt.show()

# ============================================================================
# VISUALISER TOUTES LES COUCHES (OPTIONNEL)
# ============================================================================

print("\n📊 Visualisation de toutes les couches convolutionnelles...")

n_conv_layers = len(activations)
fig, axes = plt.subplots(1, n_conv_layers, figsize=(6 * n_conv_layers, 5))

if n_conv_layers == 1:
    axes = [axes]

for idx, (layer_name, fmaps) in enumerate(activations.items()):
    fmaps_np = fmaps.cpu().numpy()[0]
    
    # Prendre la moyenne sur tous les channels pour visualisation
    fmap_mean = fmaps_np.mean(axis=0)
    
    im = axes[idx].imshow(fmap_mean, cmap='viridis', interpolation='bilinear')
    axes[idx].set_title(f'{layer_name.upper()}\n'
                        f'Shape: {fmaps_np.shape}',
                        fontsize=12, fontweight='bold')
    axes[idx].axis('off')
    plt.colorbar(im, ax=axes[idx], fraction=0.046, pad=0.04)

fig.suptitle(f'Activation Moyenne par Couche - Chiffre {sample_label}',
             fontsize=15, y=0.98, fontweight='bold')
plt.tight_layout()
plt.savefig(GRAPH_DIR / 'cnn_all_layers_activations.png', dpi=300, bbox_inches='tight')
plt.show()

print("\n" + "="*70)
print("✅ VISUALISATION DES FILTRES TERMINÉE")
print("="*70)


# 🗺️ CELLULE 8 : Réduction de Dimensionnalité (t-SNE, UMAP, PaCMAP)


In [ ]:
"""
╔══════════════════════════════════════════════════════════════════════════╗
║          VISUALISATION DES EMBEDDINGS CNN EN 2D                          ║
╚══════════════════════════════════════════════════════════════════════════╝

Objectif : Visualiser comment le CNN organise les chiffres dans l'espace latent

Méthodes comparées :
1. t-SNE : Préserve les voisinages locaux (clusters)
2. UMAP : Préserve la topologie globale et locale
3. PaCMAP : Équilibre entre structure locale et globale

Les embeddings sont extraits de la dernière couche FC (avant la sortie)
"""

print("\n" + "="*70)
print("🗺️  VISUALISATION t-SNE, UMAP, PaCMAP DES EMBEDDINGS CNN")
print("="*70)

# ============================================================================
# INSTALLER LES PACKAGES NÉCESSAIRES
# ============================================================================

print("\n📦 Vérification des packages de réduction de dimensionnalité...")

try:
    import umap
    print("   ✅ UMAP installé")
except ImportError:
    print("   📥 Installation de UMAP...")
    !pip install umap-learn -q
    import umap
    print("   ✅ UMAP installé avec succès")

try:
    import pacmap
    print("   ✅ PaCMAP installé")
except ImportError:
    print("   📥 Installation de PaCMAP...")
    !pip install pacmap -q
    import pacmap
    print("   ✅ PaCMAP installé avec succès")

from sklearn.manifold import TSNE
from sklearn.metrics import silhouette_score, davies_bouldin_score

# ============================================================================
# EXTRAIRE LES EMBEDDINGS DE LA DERNIÈRE COUCHE FC
# ============================================================================

print("\n🔄 Extraction des embeddings CNN...")

# Dictionnaire pour stocker les activations
activations_cnn = {}

def get_activation(name):
    """Hook pour capturer les activations"""
    def hook(model, input, output):
        activations_cnn[name] = output.detach()
    return hook

# Hook sur la dernière couche FC (juste avant la sortie)
target_layer = model_final.fc_layers[-1]
hook_handle = target_layer.register_forward_hook(get_activation('embeddings'))

# Forward pass sur tout le test set
model_final.eval()
with torch.no_grad():
    _ = model_final(X_test)

# Récupérer les embeddings
embeddings_cnn = activations_cnn['embeddings'].cpu().numpy()
y_true = Y_test.cpu().numpy()

# Retirer le hook
hook_handle.remove()

print(f"✅ Embeddings CNN extraits")
print(f"   Shape : {embeddings_cnn.shape}")
print(f"   Dimension des embeddings : {embeddings_cnn.shape[1]}")
print(f"   Nombre d'exemples : {embeddings_cnn.shape[0]}")

# ============================================================================
# APPLIQUER LES 3 MÉTHODES DE RÉDUCTION
# ============================================================================

print(f"\n" + "="*70)
print("🔄 APPLICATION DES RÉDUCTIONS DE DIMENSIONNALITÉ")
print("="*70)
print("⏱️  Estimation : 1-2 minutes pour les 3 méthodes\n")

results = {}

# ----------------------------------------------------------------------------
# 1. t-SNE
# ----------------------------------------------------------------------------

print(f"🔄 [1/3] t-SNE en cours...")
start = time.time()

tsne = TSNE(
    n_components=2,
    random_state=RANDOM_SEED,
    perplexity=30,
    max_iter=1000,  # ✅ Correction : max_iter au lieu de n_iter
    verbose=0
)
embeddings_tsne = tsne.fit_transform(embeddings_cnn)
time_tsne = time.time() - start

print(f"   ✅ t-SNE terminé en {time_tsne:.2f}s")
results['t-SNE'] = {'data': embeddings_tsne, 'time': time_tsne}

# ----------------------------------------------------------------------------
# 2. UMAP
# ----------------------------------------------------------------------------

print(f"\n🔄 [2/3] UMAP en cours...")
start = time.time()

reducer_umap = umap.UMAP(
    n_components=2,
    n_neighbors=15,
    min_dist=0.1,
    random_state=RANDOM_SEED,
    verbose=False
)
embeddings_umap = reducer_umap.fit_transform(embeddings_cnn)
time_umap = time.time() - start

print(f"   ✅ UMAP terminé en {time_umap:.2f}s")
results['UMAP'] = {'data': embeddings_umap, 'time': time_umap}

# ----------------------------------------------------------------------------
# 3. PaCMAP
# ----------------------------------------------------------------------------

print(f"\n🔄 [3/3] PaCMAP en cours...")
start = time.time()

reducer_pacmap = pacmap.PaCMAP(
    n_components=2,
    n_neighbors=15,
    MN_ratio=0.5,
    FP_ratio=2.0,
    random_state=RANDOM_SEED,
    verbose=False
)
embeddings_pacmap = reducer_pacmap.fit_transform(embeddings_cnn, init="pca")
time_pacmap = time.time() - start

print(f"   ✅ PaCMAP terminé en {time_pacmap:.2f}s")
results['PaCMAP'] = {'data': embeddings_pacmap, 'time': time_pacmap}

# ============================================================================
# VISUALISATION COMPARATIVE 1×3
# ============================================================================

print(f"\n📊 Génération des visualisations...")

fig, axes = plt.subplots(1, 3, figsize=(24, 7))
fig.suptitle('Embeddings CNN - Comparaison des Méthodes de Réduction\n'
             'Chaque point = 1 image du test set, couleur = chiffre réel', 
             fontsize=18, y=0.98, fontweight='bold')

methods_plot = [
    ('t-SNE', embeddings_tsne, axes[0], 'Préserve les voisinages locaux'),
    ('UMAP', embeddings_umap, axes[1], 'Préserve la topologie globale'),
    ('PaCMAP', embeddings_pacmap, axes[2], 'Équilibre local/global')
]

for method_name, data, ax, subtitle in methods_plot:
    # Scatter plot
    scatter = ax.scatter(
        data[:, 0], data[:, 1], 
        c=y_true, cmap='tab10',
        alpha=0.6, s=30, 
        edgecolors='black', linewidth=0.2
    )
    
    # Titres et labels
    ax.set_title(f'{method_name}\n{subtitle}', 
                 fontsize=14, pad=10, fontweight='bold')
    ax.set_xlabel(f'{method_name} Dimension 1', fontsize=12)
    ax.set_ylabel(f'{method_name} Dimension 2', fontsize=12)
    ax.grid(alpha=0.3, linestyle='--')
    
    # Temps d'exécution
    time_text = f"⏱️ {results[method_name]['time']:.2f}s"
    ax.text(0.02, 0.98, time_text, transform=ax.transAxes,
           fontsize=11, verticalalignment='top', weight='bold',
           bbox=dict(boxstyle='round', facecolor='white', alpha=0.9, 
                    edgecolor='black', linewidth=1.5))
    
    # Colorbar
    cbar = plt.colorbar(scatter, ax=ax, ticks=range(10))
    cbar.set_label('Chiffre', fontsize=11)

plt.tight_layout()
plt.savefig(GRAPH_DIR / 'cnn_embeddings_comparison.png', dpi=300, bbox_inches='tight')
plt.show()

# ============================================================================
# VISUALISATION PAR CLASSE (GRILLE 3×10)
# ============================================================================

print(f"\n📊 Génération des vues par classe (grille 3×10)...")

fig, axes = plt.subplots(3, 10, figsize=(28, 10))
fig.suptitle('Vue par Classe - Embeddings CNN\n'
             'Chaque sous-graphe montre où se situent les exemples d\'un chiffre spécifique', 
             fontsize=16, y=0.995, fontweight='bold')

methods_data = [
    ('t-SNE', embeddings_tsne),
    ('UMAP', embeddings_umap),
    ('PaCMAP', embeddings_pacmap)
]

for row, (method_name, data) in enumerate(methods_data):
    for digit in range(10):
        ax = axes[row, digit]
        
        # Masque pour le chiffre actuel
        mask_digit = (y_true == digit)
        
        # Afficher tous les points en gris (background)
        ax.scatter(data[~mask_digit, 0], data[~mask_digit, 1],
                   c='lightgray', alpha=0.15, s=5)
        
        # Afficher le chiffre actuel en couleur
        ax.scatter(data[mask_digit, 0], data[mask_digit, 1],
                   c=f'C{digit}', alpha=0.8, s=20, 
                   edgecolors='black', linewidth=0.2)
        
        # Labels
        if digit == 0:
            ax.set_ylabel(method_name, fontsize=12, fontweight='bold', 
                         rotation=0, labelpad=30, va='center')
        
        if row == 0:
            ax.set_title(f'Chiffre {digit}', fontsize=11, fontweight='bold')
        
        ax.set_xticks([])
        ax.set_yticks([])
        ax.grid(alpha=0.2)

plt.tight_layout()
plt.savefig(GRAPH_DIR / 'cnn_embeddings_per_class.png', dpi=300, bbox_inches='tight')
plt.show()

# ============================================================================
# ANALYSE QUANTITATIVE
# ============================================================================

print(f"\n" + "="*70)
print("📊 ANALYSE QUANTITATIVE DES EMBEDDINGS CNN")
print("="*70)

# Calculer les métriques de clustering
metrics = {}
for name, result in results.items():
    data = result['data']
    
    # Silhouette Score : mesure la cohésion et séparation des clusters
    # Valeur entre -1 et 1, plus élevé = mieux
    sil = silhouette_score(data, y_true)
    
    # Davies-Bouldin Index : mesure la compacité des clusters
    # Plus bas = mieux
    db = davies_bouldin_score(data, y_true)
    
    metrics[name] = {
        'silhouette': sil,
        'davies_bouldin': db,
        'time': result['time']
    }

# Tableau comparatif
print(f"\n{'Méthode':<10} {'Silhouette':<15} {'Davies-B':<15} {'Temps (s)':<12} {'Qualité'}")
print("=" * 75)

for method in ['t-SNE', 'UMAP', 'PaCMAP']:
    scores = metrics[method]
    sil = scores['silhouette']
    db = scores['davies_bouldin']
    t = scores['time']
    
    # Évaluation de la qualité
    if sil > 0.7:
        quality = "🥇 Excellente"
    elif sil > 0.5:
        quality = "🥈 Très bonne"
    elif sil > 0.4:
        quality = "🥉 Bonne"
    else:
        quality = "⚠️  Modérée"
    
    print(f"{method:<10} {sil:>6.4f} {'':8} {db:>6.4f} {'':8} {t:>6.2f} {'':5} {quality}")

# Identifier la meilleure méthode
best_method = max(metrics.items(), key=lambda x: x[1]['silhouette'])
fastest_method = min(metrics.items(), key=lambda x: x[1]['time'])

print(f"\n" + "=" * 75)
print("🏆 CLASSEMENT")
print("=" * 75)
print(f"🥇 Meilleure qualité : {best_method[0]} (Silhouette = {best_method[1]['silhouette']:.4f})")
print(f"⚡ Plus rapide : {fastest_method[0]} ({fastest_method[1]['time']:.2f}s)")

# ============================================================================
# GRAPHIQUE PERFORMANCE vs QUALITÉ
# ============================================================================

print(f"\n📊 Génération du graphique compromis vitesse/qualité...")

fig, ax = plt.subplots(figsize=(12, 8))

methods_list = ['t-SNE', 'UMAP', 'PaCMAP']
times = [metrics[m]['time'] for m in methods_list]
silhouettes = [metrics[m]['silhouette'] for m in methods_list]
colors = ['#ff7f0e', '#2ca02c', '#d62728']

for i, method in enumerate(methods_list):
    # Point
    ax.scatter(times[i], silhouettes[i], s=600, c=colors[i], alpha=0.7,
              edgecolors='black', linewidth=3, zorder=3)
    
    # Annotation
    ax.annotate(method, (times[i], silhouettes[i]), 
               fontsize=14, weight='bold', ha='center',
               xytext=(0, -60), textcoords='offset points',
               bbox=dict(boxstyle='round,pad=0.7', facecolor='white', 
                        edgecolor=colors[i], linewidth=2, alpha=0.9),
               arrowprops=dict(arrowstyle='->', lw=2.5, color=colors[i]))

# Labels et titre
ax.set_xlabel('Temps d\'exécution (secondes)', fontsize=14, fontweight='bold')
ax.set_ylabel('Silhouette Score (qualité de clustering)', fontsize=14, fontweight='bold')
ax.set_title('Compromis Vitesse vs Qualité - Embeddings CNN', 
             fontsize=17, pad=20, fontweight='bold')
ax.grid(alpha=0.4, linestyle='--', linewidth=1)
ax.set_xlim(left=-5)
ax.set_ylim([0, 1])

# Zones de qualité
ax.axhspan(0.7, 1.0, alpha=0.1, color='green', label='Excellente séparation (>0.7)', zorder=0)
ax.axhspan(0.5, 0.7, alpha=0.1, color='orange', label='Très bonne séparation (0.5-0.7)', zorder=0)
ax.axhspan(0.3, 0.5, alpha=0.1, color='yellow', label='Bonne séparation (0.3-0.5)', zorder=0)
ax.legend(loc='lower right', fontsize=12, framealpha=0.9)

plt.tight_layout()
plt.savefig(GRAPH_DIR / 'cnn_embeddings_speed_quality.png', dpi=300, bbox_inches='tight')
plt.show()

# ============================================================================
# INTERPRÉTATION
# ============================================================================

print(f"\n" + "="*70)
print("💡 INTERPRÉTATION DES RÉSULTATS")
print("="*70)

print(f"""
📊 Silhouette Score : {best_method[1]['silhouette']:.4f}
   → Le CNN a appris des représentations où les chiffres de même classe
     sont regroupés ensemble, et bien séparés des autres classes.

⚡ Vitesse : {fastest_method[0]} est {times[0]/times[2]:.1f}× plus rapide que t-SNE

🎯 Recommandation : 
   → Utiliser {best_method[0]} pour visualiser les embeddings CNN
   → Cette méthode offre le meilleur compromis qualité/vitesse
""")

print("\n" + "="*70)
print("✅ VISUALISATION CNN TERMINÉE")
print("="*70)
print(f"📁 Tous les graphiques sauvegardés dans : {GRAPH_DIR}")
